In [ ]:
# ==================== Relevant Imports ====================

# Torch and Torch-related modules for deep learning
import torch  # Core PyTorch functionalities
from torch import nn
torch.autograd.set_detect_anomaly(True)  # Enables anomaly detection for debugging
from transformers import ViTHybridModel, AutoModelForImageClassification
from torchvision import models  # Pretrained models

# General-purpose libraries
import os  # OS-level operations (file paths, directory handling)
import random  # Random number generation

# c libraries
from skimage import exposure  # Module for image intensity adjustment and histogram-based operations, such as contrast enhancement and histogram equalization.
import cv2  # OpenCV for image processing
from PIL import Image  # Image handling
import pydicom  # Handling DICOM medical imaging files
import pandas as pd  # Data manipulation and analysis
import numpy as np  # Numerical computations and array operations
import matplotlib.pyplot as plt  # Plotting and visualization

# Progress bar visualization
from tqdm import tqdm  # Displays progress bars for loops and tasks

# Machine learning utilities
from sklearn.model_selection import KFold  # Cross-validation splitting strategy

# Custom configurations and modules
from config import HyperP  # Hyperparameter configuration object

# ==========================================================


In [ ]:
# Instantiate the HyperP class with model-specific configurations
# 'slope_train_vit_simple' indicates the model is for slope prediction
hyp = HyperP(model_type="slope_train_vit_simple")  

# Load a range of images from a CSV file into a Pandas DataFrame
# Assumes "images.csv" contains metadata or a list of images
images_range = pd.read_csv("images.csv")

# ==================== Set Random Seed for Reproducibility ====================

# Retrieve the seed value from the HyperP object
seed = hyp.seed  

# Set the seed for Python's built-in random module
random.seed(seed)  

# Set the PYTHONHASHSEED environment variable for hash-based operations 
# to ensure deterministic results
os.environ['PYTHONHASHSEED'] = str(seed)  

# Set the seed for NumPy's random number generator
np.random.seed(seed)  

# Set the seed for PyTorch to ensure reproducibility of results
torch.manual_seed(seed)  

# Configure PyTorch's CUDA backend for deterministic results
# `deterministic = True`: Ensures consistent results by using deterministic algorithms
torch.backends.cudnn.deterministic = True  

# Disable the use of non-deterministic algorithms to avoid performance trade-offs
# `benchmark = False`: Avoids auto-tuning for optimization, ensuring repeatability
torch.backends.cudnn.benchmark = False  

In [ ]:
# Retrieve the root directory for data from the HyperP configuration object
root_path = hyp.data_folder  

# Load training metadata from a CSV file located in the data folder
# This CSV likely contains information about the training dataset
train = pd.read_csv(f'{root_path}/train.csv')  

# Filter out rows from the training data where the Patient ID matches specific values
# These Patient IDs are likely identified as problematic
train_without_badid = train[(train.Patient != 'ID00011637202177653955184') & 
                             (train.Patient != 'ID00052637202186188008618')]  


In [ ]:
# Function to generate tabular features from a DataFrame for a single patient
def get_tab(df):
    # Initialize a feature vector with the normalized age of the patient
    # Normalization is done using the mean and standard deviation of the training data's Age
    vector = [(df.Age.values[0] - train.Age.values.mean()) / train.Age.values.std()]
    
    # Add a binary encoding for the patient's gender
    # 0 for 'Male', 1 for 'Female'
    if df.Sex.values[0] == 'Male':
        vector.append(0)
    else:
        vector.append(1)
    
    # Add a one-hot encoded representation for the patient's smoking status
    # 'Never smoked' -> [0, 0]
    # 'Ex-smoker' -> [1, 1]
    # 'Currently smokes' -> [0, 1]
    # Any other status -> [1, 0]
    if df.SmokingStatus.values[0] == 'Never smoked':
        vector.extend([0, 0])
    elif df.SmokingStatus.values[0] == 'Ex-smoker':
        vector.extend([1, 1])
    elif df.SmokingStatus.values[0] == 'Currently smokes':
        vector.extend([0, 1])
    else:
        vector.extend([1, 0])
    
    # Convert the feature vector to a NumPy array and return it
    return np.array(vector)


In [ ]:
# Function to load and preprocess a medical image from a DICOM file
def get_img(path):  
    # Read the DICOM file using pydicom
    d = pydicom.dcmread(path)
    
    # Extract the pixel array from the DICOM and enhance contrast using histogram equalization
    # Resize the processed image to a fixed resolution of 384x384 using OpenCV
    output = cv2.resize(exposure.equalize_hist(d.pixel_array), (384, 384))
    
    # Return the preprocessed image
    return output

# Function to load and preprocess a mask image
def get_mask(path):
    # Open the mask image using PIL (assumes it is a standard image format like PNG or JPEG)
    mask = Image.open(path)
    
    # Convert the mask to a NumPy array and resize it to 384x384
    mask = cv2.resize(np.array(mask), (384, 384))
    
    # Reshape the mask to ensure it has dimensions 384x384 (optional step for consistency)
    return mask.reshape(384, 384)


In [ ]:
# Initialize dictionaries and list to store results
A = {}  # Dictionary to store the slopes (linear regression results) for each patient
TAB = {}  # Dictionary to store tabular features for each patient
P = []  # List to store patient IDs

# Iterate through each unique patient in the training dataset
for i, p in enumerate(tqdm(train.Patient.unique())):
    # Filter data for the current patient
    sub = train.loc[train.Patient == p, :] 
    
    # Extract Forced Vital Capacity (FVC) and Weeks (time points) for the patient
    fvc = sub.FVC.values
    weeks = sub.Weeks.values
    
    # For each patient, the code calculates the slope of FVC over time (Weeks) using linear regression, generates tabular features, and stores the results (slope, features, and patient ID) in dictionaries and a list.
    c = np.vstack([weeks, np.ones(len(weeks))]).T  # Add a column of ones for the intercept term

    a, _ = np.linalg.lstsq(c, fvc, rcond=None)[0]  # Returns the slope 'a' of the best fit line
    # ref: https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html

    A[p] = a
    TAB[p] = get_tab(sub)
    P.append(p)


100%|██████████| 176/176 [00:00<00:00, 1318.74it/s]


In [ ]:
# Define a custom dataset class for training in PyTorch
class OSICData_train(torch.utils.data.Dataset):
    """
        Custom dataset class for loading and processing training data in the OSIC dataset.
        
        This dataset class filters out bad patient IDs, loads image data, corresponding masks, slopes, and tabular features, 
        and provides a random sampling of training data.

        Attributes:
        -----------
        BAD_ID : list
            A list of patient IDs to exclude from the dataset.
        keys : list
            A list of patient IDs to include in the dataset after filtering out bad IDs.
        a : dict
            A dictionary containing slopes for each patient.
        tab : dict
            A dictionary containing tabular features for each patient.
        all_data : list
            A list of image file paths for all selected patients.
        train_data : dict
            A dictionary where the key is a patient ID, and the value is a list of corresponding image file paths.
        mask_data : dict
            A dictionary where the key is a patient ID, and the value is a list of corresponding mask file paths.
        ten_percent : list
            A random sample (50%) of the data, used for training.

        Methods:
        --------
        __len__():
            Returns the number of samples (50% of available data) in the dataset.
        
        __getitem__(idx):
            Retrieves a specific sample (image, mask, tabular features, slope, and patient ID) at the given index `idx`.
        """
    BAD_ID = ['ID00011637202177653955184', 'ID00052637202186188008618']
    def __init__(self, keys, a, tab):
            
        """
        Initializes the OSICData_train dataset.

        Parameters:
        -----------
        keys : list
            A list of patient IDs to include in the dataset.
        a : dict
            A dictionary containing slopes for each patient.
        tab : dict
            A dictionary containing tabular features for each patient.
        """
        self.keys = [k for k in keys if k not in self.BAD_ID]
        
        # Store slopes (a) and tabular features (tab)
        self.a = a
        self.tab = tab
        
        # Initialize empty lists and dictionaries for storing data
        self.all_data = []
        self.train_data = {}
        self.mask_data = {}
        
        # Loop through each patient ID in the filtered list of keys
        for p in self.keys:  
            # Get the image range (min and max slices) for the current patient from the images_range DataFrame
            properties = images_range[images_range['ID'] == p]
            min_slice = properties['min']
            max_slice = properties['max'] 

            # List the image files and mask files for the patient, excluding the first and last 15% slices
            self.train_data[p] = os.listdir(f'{root_path}/train/{p}/')[int(min_slice.iloc[0]):int(max_slice.iloc[0])] 
            self.mask_data[p] = os.listdir(f'{root_path}/mask_clear/{p}/')[int(min_slice.iloc[0]):int(max_slice.iloc[0])] 
            
            # Add each image file path to the all_data list
            for m in self.train_data[p]:
                self.all_data.append(p + '/' + m)
            
        # Sample 50% of the data randomly for training
        length = int(0.5 * len(self.all_data))
        self.ten_percent = random.sample(self.all_data, length)

    # Define the length of the dataset (number of samples)
    def __len__(self):
        """
        Returns the number of samples in the dataset.

        Returns:
        --------
        int : The number of samples in the dataset (50% of the total available data).
        """
        return len(self.ten_percent)
    
    # Method to retrieve a specific sample from the dataset
    def __getitem__(self, idx):
        """
        Returns the number of samples in the dataset.

        Returns:
        --------
        int : The number of samples in the dataset (50% of the total available data).
        """
        # Initialize empty lists to hold masks, images, slopes, and tabular features
        masks = []
        x = []
        a, tab = [], [] 
        
        # Get the patient ID and image file path for the current sample
        k = self.ten_percent[idx]
        i = k[:25]  # Extract patient ID from the image file path
       
        try:
            # Construct the corresponding mask file path by replacing the extension with .jpg
            j = k[:-4] + '.jpg'
            
            # Load the image and mask using the get_img and get_mask functions
            img = get_img(f'{root_path}/train/{k}')
            mask = get_mask(f'{root_path}/mask_clear/{j}')

            # Append the mask, image, slope, and tabular features to the respective lists
            masks.append(mask)
            x.append(img)
            a.append(self.a[i])
            tab.append(self.tab[i])
        except:
            # Print a message if there is an error loading the image or mask
            print(k, j)

        # Convert the masks, images, slopes, and tabular features into PyTorch tensors
        masks, x, a, tab = torch.tensor(np.asanyarray(masks), dtype=torch.float32), torch.tensor(np.asanyarray(x), dtype=torch.float32), torch.tensor(np.asanyarray(a), dtype=torch.float32), torch.tensor(np.asanyarray(tab), dtype=torch.float32)
        
        # Remove the extra dimension from the tabular feature tensor
        tab = torch.squeeze(tab, axis=0)

        # Return the mask, image, and tabular feature tensors, along with the slope and patient ID
        return [masks, x, tab], a, k 



In [ ]:
class OSICData_test(torch.utils.data.Dataset):
    # List of patient IDs to exclude from the dataset
    BAD_ID = ['ID00011637202177653955184', 'ID00052637202186188008618']
    
    def __init__(self, keys, a, tab):
        # Filter out bad patient IDs from the dataset
        self.keys = [k for k in keys if k not in self.BAD_ID]
        self.a = a  # Additional metadata for patients
        self.tab = tab  # Tabular data associated with patients

        self.train_data = {}  # Dictionary to store training image file names per patient
        self.mask_data = {}   # Dictionary to store corresponding mask file names per patient
        
        for p in self.keys:  # Iterate over patient IDs
            # Retrieve slice range information for the current patient
            properties = images_range[images_range['ID'] == p]
            min_slice = properties['min']
            max_slice = properties['max'] 

            # Get the total number of slices available for the patient
            p_n = len(os.listdir(f'{root_path}/train/{p}/'))

            # Select a subset of slices by removing a percentage from the beginning and end
            self.train_data[p] = os.listdir(f'{root_path}/train/{p}/')[int(hyp.strip_ct * p_n):-int(hyp.strip_ct * p_n)]
            self.mask_data[p] = os.listdir(f'{root_path}/mask_clear/{p}/')[int(hyp.strip_ct * p_n):-int(hyp.strip_ct * p_n)]

    def __len__(self):
        # Return the total number of patients in the dataset
        return len(self.keys)
    
    def __getitem__(self, idx):
        masks = []  # List to store mask images
        x = []  # List to store input images
        a, tab = [], []  # Lists to store corresponding metadata and tabular data
        
        k = self.keys[idx]  # Retrieve patient ID based on the provided index

        try:
            # Randomly select an image from the available slices for the patient
            i = np.random.choice(self.train_data[k], size=1)[0]
            j = i[:-4] + '.jpg'  # Convert filename to mask format (assuming a different extension)

            # Load the image and corresponding mask
            img = get_img(f'{root_path}/train/{k}/{i}')
            mask = get_mask(f'{root_path}/mask_clear/{k}/{j}')

            # Append the data to respective lists
            masks.append(mask)
            x.append(img)
            a.append(self.a[k])
            tab.append(self.tab[k])
        except:
            print(k, i)  # Print the patient ID and image filename in case of an error

        # Convert lists to PyTorch tensors
        masks = torch.tensor(np.asanyarray(masks), dtype=torch.float32)
        x = torch.tensor(np.asanyarray(x), dtype=torch.float32)
        a = torch.tensor(np.asanyarray(a), dtype=torch.float32)
        tab = torch.tensor(np.asanyarray(tab), dtype=torch.float32)
        
        # Remove extra dimensions from tabular data
        tab = torch.squeeze(tab, axis=0)

        return [masks, x, tab], a, k  # Return the data along with the patient ID


In [11]:
# Identity class
class Identity(nn.Module):
    # credit: ptrblck
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [12]:
class UnlearnableIdentityConvModel(nn.Module):
    def __init__(self):
        super(UnlearnableIdentityConvModel, self).__init__()
        self.conv = nn.Conv2d(
            in_channels=64,      # Number of input channels
            out_channels=64,     # Number of output channels
            kernel_size=1,       # 1x1 convolution
            stride=1,            # Stride of 1
            padding=0,           # No padding
            bias=False           # No bias term needed
        )
        
        # Initialize weights to be identity
        self.conv.weight.data = torch.eye(64).view(64, 64, 1, 1)
        
        # Freeze the weights
        self.conv.weight.requires_grad = False

    def forward(self, x):
        return self.conv(x)

In [14]:
# device
gpu = torch.device(f"cuda:{hyp.gpu_index}" if torch.cuda.is_available() else "cpu")
device = gpu

In [15]:
class Sparsemax(nn.Module):
    """
    This class is adapted from the TabNet implementation:
    https://github.com/dreamquark-ai/tabnet/blob/develop/pytorch_tabnet/tab_network.py

    Sparsemax activation function for neural networks.
    """
    def __init__(self, dim=None):
        super(Sparsemax, self).__init__()
        self.dim = -1 if dim is None else dim

    def forward(self, input):
        input = input.transpose(0, self.dim)
        original_size = input.size()
        input = input.reshape(input.size(0), -1)
        input = input.transpose(0, 1)
        dim = 1

        number_of_logits = input.size(dim)
        
        input = input - torch.max(input, dim=dim, keepdim=True)[0].expand_as(input)
        zs = torch.sort(input=input, dim=dim, descending=True)[0]
        range = torch.arange(start=1, end=number_of_logits + 1, device=device,step=1, dtype=input.dtype).view(1, -1)
        range = range.expand_as(zs)

        bound = 1 + range * zs
        cumulative_sum_zs = torch.cumsum(zs, dim)
        is_gt = torch.gt(bound, cumulative_sum_zs).type(input.type())
        k = torch.max(is_gt * range, dim, keepdim=True)[0]
        zs_sparse = is_gt * zs
        taus = (torch.sum(zs_sparse, dim, keepdim=True) - 1) / k
        taus = taus.expand_as(input)
        self.output = torch.max(torch.zeros_like(input), input - taus)
        output = self.output
        output = output.transpose(0, 1)
        output = output.reshape(original_size)
        output = output.transpose(0, self.dim)
        return output
    def backward(self, grad_output):
        dim = 1
        nonzeros = torch.ne(self.output, 0)
        sum = torch.sum(grad_output * nonzeros, dim=dim) / torch.sum(nonzeros, dim=dim)
        self.grad_input = nonzeros * (grad_output - sum.expand_as(grad_output))
        return self.grad_input

In [16]:
def initialize_non_glu(module,inp_dim,out_dim):
     """
    Initialize the weights of a given module using Xavier normal initialization.

    Reference:
    Adapted from TabNet implementation:
    https://github.com/dreamquark-ai/tabnet/blob/develop/pytorch_tabnet/tab_network.py

    Args:
        module (torch.nn.Module): The module whose weights need initialization.
        inp_dim (int): Input dimension of the layer.
        out_dim (int): Output dimension of the layer.

    Returns:
        None
    """
     gain = np.sqrt((inp_dim+out_dim)/np.sqrt(4*inp_dim))
     torch.nn.init.xavier_normal_(module.weight, gain=gain)

In [17]:
class GBN(nn.Module):
    """
    Ghost Batch Normalization (GBN).
    Adapted from TabNet implementation:
    https://github.com/dreamquark-ai/tabnet/blob/develop/pytorch_tabnet/tab_network.py
    """
    def __init__(self, inp, vbs=128, momentum=0.01):
        super().__init__()
        self.bn = nn.BatchNorm1d(inp, momentum=momentum)
        self.vbs = vbs
    
    def forward(self, x):
        chunk = torch.chunk(x, max(1, x.size(0) // self.vbs), 0)
        res = [self.bn(y) for y in chunk]
        return torch.cat(res, 0)

class GLU(nn.Module):
    """ 
    Gated Linear Unit (GLU) activation 
    Adapted from TabNet implementation:
    https://github.com/dreamquark-ai/tabnet/blob/develop/pytorch_tabnet/tab_network.py
    """
    def __init__(self, inp_dim, out_dim, fc=None, vbs=128):
        super().__init__()
        self.fc = fc if fc else nn.Linear(inp_dim, out_dim * 2)
        self.bn = GBN(out_dim * 2, vbs=vbs)
        self.od = out_dim
    
    def forward(self, x):
        x = self.bn(self.fc(x))
        return x[:, :self.od] * torch.sigmoid(x[:, self.od:])

class FeatureTransformer(nn.Module):
    """ 
    Feature transformation block using GLU layers 
    Adapted from TabNet implementation:
    https://github.com/dreamquark-ai/tabnet/blob/develop/pytorch_tabnet/tab_network.py
    """
    def __init__(self, inp_dim, out_dim, shared, n_ind, vbs=128):
        super().__init__()
        self.shared = nn.ModuleList([GLU(inp_dim, out_dim, shared[0], vbs=vbs)]) if shared else None
        self.independ = nn.ModuleList([GLU(out_dim, out_dim, vbs=vbs) for _ in range(n_ind)])
        self.scale = torch.sqrt(torch.tensor([.5], device=device))
    
    def forward(self, x):
        if self.shared:
            x = sum(glu(x) for glu in self.shared) * self.scale
        for glu in self.independ:
            x = (x + glu(x)) * self.scale
        return x

class AttentionTransformer(nn.Module):
    """ 
    Attention-based feature selection 
    Adapted from TabNet implementation:
    https://github.com/dreamquark-ai/tabnet/blob/develop/pytorch_tabnet/tab_network.py
    """
    def __init__(self, inp_dim, out_dim, relax, vbs=128):
        super().__init__()
        self.fc = nn.Linear(inp_dim, out_dim)
        self.bn = GBN(out_dim, vbs=vbs)
        self.r = torch.tensor([relax], device=device)
    
    def forward(self, a, priors):
        a = self.bn(self.fc(a))
        mask = torch.sigmoid(a * priors)
        priors = priors * (self.r - mask)
        return mask

class DecisionStep(nn.Module):
    """ 
    Decision step combining feature transformation and attention 
    Adapted from TabNet implementation:
    https://github.com/dreamquark-ai/tabnet/blob/develop/pytorch_tabnet/tab_network.py
    """
    def __init__(self, inp_dim, n_d, n_a, shared, n_ind, relax, vbs=128):
        super().__init__()
        self.fea_tran = FeatureTransformer(inp_dim, n_d + n_a, shared, n_ind, vbs)
        self.atten_tran = AttentionTransformer(n_a, inp_dim, relax, vbs)
    
    def forward(self, x, a, priors):
        mask = self.atten_tran(a, priors)
        loss = (-mask * torch.log(mask + 1e-10)).mean()
        x = self.fea_tran(x * mask)
        return x, loss

In [ ]:
class TabCT(nn.Module):
    def __init__(self, cnn,num_features=4, feature_dim=40, output_dim=20, num_decision_steps=3,  # 2 boud
                    relaxation_factor=1, batch_momentum=0.1, epsilon=0.00001, vgg_npy_path = None
                    , n_shared=2, n_d=64, n_a=64, n_ind=2, n_steps=4,relax=1.2,vbs=128):
        super(TabCT, self).__init__()
        
        if n_shared>0:
            self.shared = nn.ModuleList()
            self.shared.append(nn.Linear(num_features,2*(n_d+n_a)))
            for x in range(n_shared-1):
                self.shared.append(nn.Linear(n_d+n_a,2*(n_d+n_a)))
        else:
            self.shared=None
        self.first_step = FeatureTransformer(num_features,n_d+n_a,self.shared,n_ind) 
        self.steps = nn.ModuleList()
        for x in range(n_steps-1):
            self.steps.append(DecisionStep(num_features,n_d,n_a,self.shared,n_ind,relax,vbs))
        self.fc_tab = nn.Linear(n_d,output_dim)
        self.bn = nn.BatchNorm1d(num_features)
        self.n_d = n_d

        # CT features
        cnn_dict = {'vit_base_patch16lung0' :None, 'vit_b_16':None, 'vgg16':models.vgg16, 'resnet18': models.resnet18, 'resnet34': models.resnet34, 'resnet50': models.resnet50,
                   'resnet101': models.resnet101, 'resnet152': models.resnet152, 'resnext50': models.resnext50_32x4d,
                   'resnext101': models.resnext101_32x8d}
        '''
        fully conected 
        self.fullyconected = nn.Linear(5, 25)
        '''



      


        self.num_features = num_features
        self.feature_dim = feature_dim
        self.output_dim = output_dim
        self.num_decision_steps = num_decision_steps
        self.relaxation_factor = relaxation_factor
        self.batch_momentum = batch_momentum
        # self.virtual_batch_size = virtual_batch_size
        # self.num_classes = num_classes
        self.epsilon = epsilon
        # feature dim
        self.out_dict = {'vit_base_patch16lung0':768, 'vit_b_16': 768, 'resnet18': 512, 'resnet34': 512, 'resnet50': 2048, 'resnet101': 2048, 'resnet152': 2048,
                         'resnext50': 2048, 'resnext101': 2048, "efnb0": 1280, "efnb1": 1280, "efnb2": 1408, 
                          "efnb3": 1536, "efnb4": 1792, "efnb5": 2048, "efnb6": 2304, "efnb7": 2560, "vgg16": 512}
        
        self.n_tab = hyp.n_tab # n tabular features
        
        # efficient net b0 to b7
        
        if cnn in cnn_dict.keys(): # resnet or resnext or vgg16
            if cnn == 'vgg16':
                # load vgg16 model badan inja be khatere none noudan ye if bezar

                self.ct_cnn = cnn_dict[cnn](pretrained = True).features[2:]
                self.conv = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                self.W = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty((64, 3, 3, 3)), mean=0, std=0.01))
                self.B = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty(64), mean=0, std=0.01))

            elif cnn == 'vit_b_16':


                # model
                self.con1 = ViTHybridModel.from_pretrained("google/vit-hybrid-base-bit-384")
                # print("model")          


                # change configuration 
                self.input_channel_after_mul = 64
                self.input_size_after_mul = 192

                self.con1.config.num_channels = self.input_channel_after_mul
                self.con1.config.image_size = self.input_size_after_mul 

                self.con1.embeddings.patch_embeddings.num_channels = self.input_channel_after_mul
                self.con1.embeddings.patch_embeddings.image_size = (self.input_size_after_mul , self.input_size_after_mul)

                self.con1.embeddings.patch_embeddings.backbone.bit.config.num_channels = self.input_channel_after_mul

                self.con1.embeddings.patch_embeddings.backbone.bit.embedder.num_channels = self.input_channel_after_mul

                # First CNN Layer
                self.conv = self.con1.embeddings.patch_embeddings.backbone.bit.embedder.convolution
                # print("First CNN Layer")

                # First Mask Layer
                self.W = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty((64, 3, 7, 7)), mean=0, std=0.01))
                self.B = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty(64), mean=0, std=0.01))
                self.mask = self.con1.embeddings.patch_embeddings.backbone.bit.embedder.convolution
                self.mask.weight = self.W
                self.mask.bias = self.B
                # print("First Mask Layer")

                # Rest of Embeddings
                self.embeddings = self.con1.embeddings
                self.embeddings.patch_embeddings.backbone.bit.embedder.convolution = UnlearnableIdentityConvModel()
                # print(self.embeddings)
                # print("Rest of Embeddings")

                # Encoder(ViT), layernorm, pooler layer
                self.ct_cnn = self.con1.encoder
                # print(self.ct_cnn)
                self.norm = self.con1.layernorm
                self.pooler = self.con1.pooler
                # print("Encoder(ViT), layernorm, pooler layer")



            elif cnn == 'vit_base_patch16lung0':
                self.con1 = AutoModelForImageClassification.from_pretrained("DunnBC22/vit-base-patch16-224-in21k_lung_and_colon_cancer").vit
                self.conv_layer = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3, stride=3, padding=80)
                self.conv = self.con1.embeddings.patch_embeddings.backbone.embedder# nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))# self.ct_cnn.conv_proj 
                self.ct_cnn = self.con1.encoder

                self.W = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty((768, 3, 3, 3)), mean=0, std=0.01))
                self.B = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty(768), mean=0, std=0.01))
                self.mask = self.con1.embeddings
                self.mask.patch_embeddings.projection.weight = self.W
                self.mask.patch_embeddings.projection.bias = self.B

                self.norm = self.con1.layernorm
                self.pooler = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k").pooler
            else:   
                self.ct_cnn = cnn_dict[cnn](pretrained = True)
                
                # make single channel
                self.conv = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                self.ct_cnn.conv1 = nn.Identity()
                self.W = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty((64, 3, 3, 3)), mean=0, std=0.01))
                self.B = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty(64), mean=0, std=0.01))
                # self.ct_cnn.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
                
                # remove the fc layer/ add a simple linear layer
                self.ct_cnn.fc = nn.Linear(self.out_dict[cnn], hyp.cnn_dim)   # mapped to 64 dimensions, Identity()
            
        else:
            raise ValueError("cnn not recognized")
        
        # second feature extractor
        self.ct_cnn_s = models.resnet18(pretrained = True)
        self.conv_s = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.ct_cnn_s.conv1 = nn.Identity()
        self.W_s = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty((64, 3, 7, 7)), mean=0, std=0.01))
        self.B_s = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty(64), mean=0, std=0.01))
        self.mask_s = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.mask_s.weight = self.W_s
        self.mask_s.bias = self.B_s
        self.dropout = nn.Dropout(p=0.3)
        print("second feature extractor")
        
        self.fc_inter = nn.Linear(hyp.cnn_dim_s + self.n_tab + hyp.cnn_dim, hyp.fc_dim) 

        self.BN_fc_inter = nn.BatchNorm1d(hyp.fc_dim,
                                        momentum=self.batch_momentum)

        self.fc = nn.Linear(hyp.fc_dim, 1)


    def forward(self, x_ct, x_tab, masks):

        x_temp = self.bn(x_tab)
        x_a = self.first_step(x_temp)[:,self.n_d:]
        loss = torch.zeros(1).to(x_temp.device)
        out = torch.zeros(x_temp.size(0),self.n_d).to(x_temp.device)
        priors = torch.ones(x_temp.shape).to(x_temp.device)
        for step in self.steps:
            x_te,l = step(x_temp,x_a,priors)
            out += nn.functional.relu(x_te[:,:self.n_d])
            x_a = x_te[:,self.n_d:]
            loss += l
        # all_loss = []
        # self.all_loss.append(loss)
        # print("tabular finished")
        # 1 + 1 + 1
        x_ct = torch.cat((x_ct, torch.cat((x_ct, x_ct), 1)), 1)
        masks = torch.cat((masks, torch.cat((masks, masks), 1)), 1)
        # print("input concatenate")


        feature_map = self.conv(x_ct) # ViT
        feature_map_s = self.conv_s(x_ct) # CNN
        # print("first layer image")

        
        relevance_map_s = self.mask_s(masks) # CNN
        relevance_map = self.mask(masks)  #self.B # ViT
        # print("first layer mask")

        # multiple element-wise
        ct_att = torch.mul(feature_map, relevance_map) # ViT
        ct_att_s = torch.mul(feature_map_s, relevance_map_s) # CNN
        # print("multiply both")

        ct_f_s = self.ct_cnn_s(ct_att_s) # CNN 
        # print("rest of CNN")

        # ViT
        # print(ct_att.size())
        # print(ct_att.shape)
        ct_f = self.embeddings(ct_att)
        # print("embeddings")
        ct_f = self.ct_cnn(ct_f)
        # print("ct_cnn")
        # print(ct_f)
        # print(ct_f['last_hidden_state'].size())
        # print(type(ct_f))
        ct_f = self.norm(ct_f['last_hidden_state']) # ct features
        # print("norm")
        ct_f = self.pooler(ct_f)
        # print("pooler")
        # print("rest of ViT")

        # concatenate
        x = torch.cat((ct_f_s, self.fc_tab(out)), -1) # concat on last axis output_aggregated  # changed
        x = torch.cat((ct_f, x), -1) # concat on last axis #changed
        # print("concatenate outputs")

        # dropout
        x = self.dropout(x)

        x = self.fc_inter(x)

        x = self.fc(x)
        
        return x, loss

In [ ]:
class TabCT(nn.Module):
    def __init__(self, cnn, num_features=4, output_dim=20, 
                 n_shared=2, n_d=64, n_a=64, n_ind=2, n_steps=4, relax=1.2, vbs=128):
        super(TabCT, self).__init__()

        # If shared layers are defined, create them
        if n_shared > 0:
            self.shared = nn.ModuleList()
            # First shared layer from num_features to 2*(n_d+n_a)
            self.shared.append(nn.Linear(num_features, 2*(n_d + n_a)))
            for _ in range(n_shared - 1):
                # Additional shared layers with dimensions (n_d+n_a)
                self.shared.append(nn.Linear(n_d + n_a, 2*(n_d + n_a)))
        else:
            self.shared = None
        
        # First feature transformer step
        self.first_step = FeatureTransformer(num_features, n_d + n_a, self.shared, n_ind)
        
        # Additional decision steps
        self.steps = nn.ModuleList()
        for _ in range(n_steps - 1):
            # Decision steps apply transformation to feature representation
            self.steps.append(DecisionStep(num_features, n_d, n_a, self.shared, n_ind, relax, vbs))
        
        # Fully connected layer for tabular features to output_dim
        self.fc_tab = nn.Linear(n_d, output_dim)
        
        # Batch normalization for tabular features
        self.bn = nn.BatchNorm1d(num_features)
        
        # Save the dimensions of features (n_d)
        self.n_d = n_d

        # # CNN Model dictionary with different backbone models (e.g., resnet, vgg, etc.)
        # cnn_dict = {'vit_base_patch16lung0': None, 'vit_b_16': None, 'vgg16': models.vgg16, 
        #             'resnet18': models.resnet18, 'resnet34': models.resnet34, 
        #             'resnet50': models.resnet50, 'resnet101': models.resnet101, 
        #             'resnet152': models.resnet152, 'resnext50': models.resnext50_32x4d, 
        #             'resnext101': models.resnext101_32x8d}
        
        # Dictionary for output dimensions of different models
        self.out_dict = {'vit_base_patch16lung0': 768, 'vit_b_16': 768, 'resnet18': 512, 'resnet34': 512, 
                         'resnet50': 2048, 'resnet101': 2048, 'resnet152': 2048, 'resnext50': 2048, 
                         'resnext101': 2048, "efnb0": 1280, "efnb1": 1280, "efnb2": 1408, "efnb3": 1536, 
                         "efnb4": 1792, "efnb5": 2048, "efnb6": 2304, "efnb7": 2560, "vgg16": 512}

        # Number of tabular features
        self.n_tab = hyp.n_tab  # n tabular features

        # Initialize the appropriate CNN model based on input 'cnn'
        # if cnn in cnn_dict.keys():
            # if cnn == 'vgg16':
            #     # For VGG16, customize CNN architecture by extracting layers
            #     self.ct_cnn = cnn_dict[cnn](pretrained=True).features[2:]
            #     self.conv = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            #     self.W = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty((64, 3, 3, 3)), mean=0, std=0.01))
            #     self.B = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty(64), mean=0, std=0.01))
            # elif cnn == 'vit_b_16':
                # For Vision Transformer (ViT), use pre-trained model and modify configuration
        self.con1 = ViTHybridModel.from_pretrained("google/vit-hybrid-base-bit-384")
        self.input_channel_after_mul = 64
        self.input_size_after_mul = 192
        # Adjust ViT configuration to work with our input size and channels
        self.con1.config.num_channels = self.input_channel_after_mul
        self.con1.config.image_size = self.input_size_after_mul
        self.ct_cnn = self.con1.encoder
        self.norm = self.con1.layernorm
        self.pooler = self.con1.pooler
            # elif cnn == 'vit_base_patch16lung0':
            #     # Load a pre-trained ViT model for lung and colon cancer classification
            #     self.con1 = AutoModelForImageClassification.from_pretrained("DunnBC22/vit-base-patch16-224-in21k_lung_and_colon_cancer").vit
            #     self.ct_cnn = self.con1.encoder
            #     self.norm = self.con1.layernorm
            #     self.pooler = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k").pooler
            # else:
            #     # For other models, initialize with pre-trained weights
            #     self.ct_cnn = cnn_dict[cnn](pretrained=True)
            #     self.conv = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            #     self.ct_cnn.conv1 = nn.Identity()
            #     self.W = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty((64, 3, 3, 3)), mean=0, std=0.01))
            #     self.B = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty(64), mean=0, std=0.01))
            #     self.ct_cnn.fc = nn.Linear(self.out_dict[cnn], hyp.cnn_dim)

        # Secondary feature extractor (ResNet18)
        self.ct_cnn_s = models.resnet18(pretrained=True)
        self.conv_s = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.ct_cnn_s.conv1 = nn.Identity()
        self.W_s = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty((64, 3, 7, 7)), mean=0, std=0.01))
        self.B_s = nn.Parameter(torch.nn.init.trunc_normal_(torch.empty(64), mean=0, std=0.01))
        self.mask_s = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.mask_s.weight = self.W_s
        self.mask_s.bias = self.B_s

        # Dropout layer for regularization
        self.dropout = nn.Dropout(p=0.3)

        # Intermediate fully connected layer
        self.fc_inter = nn.Linear(hyp.cnn_dim_s + self.n_tab + hyp.cnn_dim, hyp.fc_dim)
    

        # Final fully connected layer to predict output
        self.fc = nn.Linear(hyp.fc_dim, 1)

    def forward(self, x_ct, x_tab, masks):
        """
        Forward pass of the TabCT model.
        x_ct: Input CT scan images
        x_tab: Input tabular features (e.g., clinical data)
        masks: Masks for attention mechanisms in the model
        """
        # Batch normalization for tabular features
        x_temp = self.bn(x_tab)
        
        # First feature transformation step
        x_a = self.first_step(x_temp)[:, self.n_d:]
        
        # Initialize loss and output tensors
        loss = torch.zeros(1).to(x_temp.device)
        out = torch.zeros(x_temp.size(0), self.n_d).to(x_temp.device)
        priors = torch.ones(x_temp.shape).to(x_temp.device)
        
        # Process through additional decision steps
        for step in self.steps:
            x_te, l = step(x_temp, x_a, priors)
            out += nn.functional.relu(x_te[:, :self.n_d])
            x_a = x_te[:, self.n_d:]
            loss += l
        
        # Concatenate CT scan features and duplicate masks for processing
        x_ct = torch.cat((x_ct, torch.cat((x_ct, x_ct), 1)), 1)
        masks = torch.cat((masks, torch.cat((masks, masks), 1)), 1)
        
        # Pass the concatenated input through the first CNN layer
        feature_map = self.conv(x_ct)  # ViT model
        feature_map_s = self.conv_s(x_ct)  # CNN model

        # Apply attention mechanism using masks
        relevance_map_s = self.mask_s(masks)  # CNN mask
        relevance_map = self.mask(masks)  # ViT mask

        # Apply element-wise multiplication (attention) to the feature maps
        ct_att = torch.mul(feature_map, relevance_map)  # ViT
        ct_att_s = torch.mul(feature_map_s, relevance_map_s)  # CNN
        
        # Process CNN feature map
        ct_f_s = self.ct_cnn_s(ct_att_s)

        # Process ViT feature map through embeddings, encoder, normalization, and pooler
        ct_f = self.embeddings(ct_att)
        ct_f = self.ct_cnn(ct_f)
        ct_f = self.norm(ct_f['last_hidden_state'])  # ViT features
        ct_f = self.pooler(ct_f)

        # Concatenate both CNN and ViT outputs
        x = torch.cat((ct_f_s, self.fc_tab(out)), -1)  # Concatenated feature map
        x = torch.cat((ct_f, x), -1)  # Concatenate final feature map

        # Apply dropout for regularization
        x = self.dropout(x)

        # Final fully connected layers
        x = self.fc_inter(x)
        x = self.fc(x)
        
        return x, loss


In [19]:
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score
def score(fvc_true, fvc_pred, sigma):
    sigma_clip = np.maximum(sigma, 70)
    delta = np.abs(fvc_true - fvc_pred)
    delta = np.minimum(delta, 1000)
    sq2 = np.sqrt(2)
    metric = (delta / sigma_clip)*sq2 + np.log(sigma_clip* sq2)
    return np.mean(metric)


def score_avg(p, a): # patient id, predicted a
    percent_true = train.Percent.values[train.Patient == p]
    fvc_true = train.FVC.values[train.Patient == p]
    weeks_true = train.Weeks.values[train.Patient == p]

    fvc = a * (weeks_true - weeks_true[0]) + fvc_true[0]
    percent = percent_true[0] - a * abs(weeks_true - weeks_true[0])
    return score(fvc_true, fvc, percent)

def rmse_avg(p, a): # patient id, predicted a
    percent_true = train.Percent.values[train.Patient == p]
    fvc_true = train.FVC.values[train.Patient == p]
    weeks_true = train.Weeks.values[train.Patient == p]

    fvc = a * (weeks_true - weeks_true[0]) + fvc_true[0]
    return root_mean_squared_error(fvc_true, fvc)



def smape(targets, outs):
    denominator = (np.abs(targets) + np.abs(outs)) / 2
    ape = np.abs(targets - outs) / denominator
    return np.mean(ape) * 100

# def test_r2_score(p, a):
#     fvc_true = train.FVC.values[train.Patient == p]
# test_r2_score = r2_score(p.numpy(), pred_a[p].numpy())
# test_mape_score = torch.mean(torch.abs((p - pred_a[p]) / p)) * 100
# test_mae_score = torch.mean(torch.abs(pred_a[p] - p))
# test_smape_score = smape(p, pred_a[p])

def plot_figures(figures, nrows = 1, ncols=1):
    """Plot a dictionary of figures.

    Parameters
    ----------
    figures : <title, figure> dictionary
    ncols : number of columns of subplots wanted in the display
    nrows : number of rows of subplots wanted in the figure
    """

    fig, axeslist = plt.subplots(ncols=ncols, nrows=nrows)
    for ind,title in enumerate(figures):
        axeslist.ravel()[ind].imshow(figures[title], cmap=plt.gray())
        axeslist.ravel()[ind].set_title(title)
        axeslist.ravel()[ind].set_axis_off()
    plt.tight_layout() # optional

In [20]:
# hyperparams
result_dir = hyp.results_dir

# training only resnet models on gpu 0
train_models = hyp.train_models 

In [21]:
from sklearn.model_selection import train_test_split
train_s, test = train_test_split(P, test_size=0.2, random_state=57)
len(train_s)

140

In [ ]:
nfold = hyp.nfold # hyper

# removing noisy data
P = [p for p in P if p not in ['ID00011637202177653955184', 'ID00052637202186188008618']] # in bayad bere bala

for model in train_models:
    log = open(f"{result_dir}/september8th/{model}simplestep2hybrid.txt", "a+")
    kfold =KFold(n_splits=nfold)
    
    ifold = 0
    min_sq = {}

    for train_index, valid_index in kfold.split(train_s):  
        
        p_train = np.array(P)[train_index] 
        p_valid = np.array(P)[valid_index] 
        print(len(p_train))
        osic_train = OSICData_train(p_train, A, TAB)
        train_loader = torch.utils.data.DataLoader(osic_train, batch_size=hyp.batch_size, shuffle=True, num_workers=hyp.num_workers, drop_last=True)

        osic_val = OSICData_test(p_valid, A, TAB)
        val_loader = torch.utils.data.DataLoader(osic_val, batch_size=hyp.batch_size, shuffle=True, num_workers=hyp.num_workers)
        print(len(osic_train))
        print(len(train_loader))
        print(len(val_loader))

        tabct = TabCT(cnn = model).to(gpu)
        print(f"creating {model}")
        print(f"fold: {ifold}")
        log.write(f"fold: {ifold}\n")

        

        n_epochs = hyp.n_epochs # max 30 epochs, patience 5, find the suitable epoch number for later final training

        best_epoch = n_epochs # 30


        optimizer = torch.optim.AdamW(tabct.parameters())
        criterion = torch.nn.L1Loss()

        max_score = 99999999.0000 # here, max score ]= minimum score
        tot_rmse = []
        for epoch in range(n_epochs):  # loop over the dataset multiple times
            running_loss = 0.0
            tabct.train()

            tabular_loss = []
            for i, data in enumerate(tqdm(train_loader, 0)):

                [mask, x, t], a, _ = data

                x = x.to(gpu)
                mask = mask.to(gpu)
                t = t.to(gpu)
                a = a.to(gpu)
                # print(x)
                # print(t)
                # print(mask)
                # t hanoun tabe
                # ultimate = ultimate.to(gpu)
                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward + backward + optimize
                outputs, tab_loss = tabct(x, t, mask) # here
                # print(outputs.size())
                tabular_loss.append(tab_loss)
                loss = criterion(outputs, a)
                loss.backward()
                optimizer.step()

                # print statistics
                running_loss += loss.item()
            print(f"tabular loss: {tabular_loss}")
            print(f"epoch {epoch+1} train: {running_loss}")
            log.write(f"epoch {epoch+1} train: {running_loss}\n")


            running_loss = 0.0
            pred_a = {}
            tabct.eval()
            tabular_loss = []
            for i, data in enumerate(tqdm(val_loader, 0)):

                [mask, x, t], a, pid = data

                x = x.to(gpu)
                mask = mask.to(gpu)
                t = t.to(gpu)
                a = a.to(gpu)

                # forward
                outputs, tab_loss = tabct(x, t, mask)
                loss = criterion(outputs, a)
                tabular_loss.append(tab_loss)
                pids = pid
                preds_a = outputs.detach().cpu().numpy().flatten()

                for j, p_d in enumerate(pids):
                    pred_a[p_d] = preds_a[j]

               


                # print statistics
                running_loss += loss.item()
            print(tabular_loss)
            print(f"epoch {epoch+1} val: {running_loss}")
            log.write(f"epoch {epoch+1} val: {running_loss}\n")
            # score calculation
            # print(pred_a)
            # print(len(pred_a))
            # print(p_test)
            # print(len(p_test))

            # totals
            tot_r2_score = []
            tot_mape_score = []
            tot_mae_score = []
            tot_smape_score = []

            # everyone
            score_v = 0.
            rmse = 0.
            test_r2_score = []
            test_mape_score = []
            test_mae_score = []
            test_smape_score = []
            print(len(p_valid))
            # fvcs_true = []
            # fvcs_pred = []
            for p in p_valid:
                score_v += (score_avg(p, pred_a[p]))/len(p_valid)
                rmse += (rmse_avg(p, pred_a[p]))/len(p_valid)
                fvc_true = train.FVC.values[train.Patient == p]
                weeks_true = train.Weeks.values[train.Patient == p]
                fvc_pred = pred_a[p] * (weeks_true - weeks_true[0]) + fvc_true[0]

                test_r2_score.append(r2_score(fvc_true, fvc_pred))
                test_mape_score.append(np.mean(np.abs((fvc_true - fvc_pred) / fvc_true)) * 100)
                test_mae_score.append(np.mean(np.abs(fvc_pred - fvc_true)))
                test_smape_score.append(smape(fvc_true, fvc_pred))
            #------------------------
            tot_rmse.append(rmse)
            tot_r2_score.append(np.asanyarray(test_r2_score))
            tot_mape_score.append(np.asanyarray(test_mape_score))
            tot_mae_score.append(np.asanyarray(test_mae_score))
            tot_smape_score.append(np.asanyarray(test_smape_score))
            #------------------------

            print("this is rmse")
            print(tot_rmse)
            print("this is r2")
            print(tot_r2_score)
            print("this is mape")
            print(tot_mape_score)
            print("this is mae")
            print(tot_mae_score)
            print("this is smape")
            print(tot_smape_score)
            print(f"val score: {score_v}")
            log.write(f"val score: {score_v}\n")
            log.write(f"val rmse: {rmse}\n")

            if score_v <= max_score:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': tabct.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'score': score_v
                    }, f"{result_dir}/september8th/{model}simplestep2hybrid.tar")
                max_score = score_v
                best_epoch = epoch + 1
        min_sq[ifold] = np.array(tot_rmse)
        # print("all mean square error")
        # print(min_sq)
        ifold += 1
        # destroy model
        del tabct
        torch.cuda.empty_cache()


# # ref: https://www.kaggle.com/miklgr500/linear-decay-based-on-resnet-cnn
# # https://pytorch.org/docs/stable/index.html



112
6021
752
4


c:\Users\Vison\Documents\Users\Dolatabadi\Dolatabadi\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ViTHybridModel were not initialized from the model checkpoint at google/vit-hybrid-base-bit-384 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Vison\Documents\Users\Dolatabadi\Dolatabadi\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Vison\Documents\Users\Dolatabadi\Dolatabadi\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments oth

second feature extractor
creating vit_b_16
fold: 0


100%|██████████| 752/752 [17:11<00:00,  1.37s/it]


tabular loss: [tensor([0.9053], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9045], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.8873], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9140], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.8982], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.8871], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9089], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9055], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.8971], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.8963], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.8953], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.8857], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.8986], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.8960], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9097], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.8925], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9209], device='cuda:0', grad_fn

...

[tensor([0.9481], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.8989], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9200], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9661], device='cuda:0', grad_fn=<AddBackward0>)]
epoch 4 val: 17.34344482421875
28
this is rmse
[438.01428499508273, 169.8022371505267, 171.17450376187284, 166.6154039290472]
this is r2
[array([ 4.24330824e-01, -2.04949592e+00,  6.75648700e-01, -8.13549538e+00,
       -1.86652585e+00, -8.03668773e-01, -1.30240104e+00,  4.07865476e-01,
        1.03963906e-01, -3.43958927e-01,  5.44709895e-01, -2.00729890e-01,
       -4.88938617e-01,  7.03950621e-01, -5.78117808e-01,  2.28160213e-01,
        9.15982702e-02, -1.48297494e-01, -3.70802663e-03, -4.96853454e+00,
       -4.72005080e-01,  3.40591667e-03, -4.34022614e+00,  5.82050751e-01,
       -3.88147039e-01,  1.60204700e-01,  4.16616166e-01, -8.18216957e-01])]
this is mape
[array([ 6.34237186,  6.97313145,  1.16412071,  7.42079381,  8.1976499 ,
        8.50

100%|██████████| 873/873 [24:13<00:00,  1.66s/it]


tabular loss: [tensor([0.9175], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9007], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.8975], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9234], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9004], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.8997], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9181], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9194], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9304], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9164], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9120], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9105], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9206], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9113], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9028], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9033], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9187], device='cuda:0', grad_fn

100%|██████████| 4/4 [00:03<00:00,  1.03it/s]


[tensor([0.9602], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9676], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9704], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.7651], device='cuda:0', grad_fn=<AddBackward0>)]
epoch 5 val: 17.895178079605103
28
this is rmse
[438.01428499508273, 169.8022371505267, 171.17450376187284, 166.6154039290472, 163.59738854633414]
this is r2
[array([ 0.44216396, -1.63922269,  0.7597417 , -7.65143159, -1.97504247,
       -1.22118908, -0.76424286,  0.5928941 ,  0.29369711, -0.45180583,
        0.62663751, -0.40975609, -0.23076864,  0.6991609 , -0.7520485 ,
        0.58189572,  0.28128452, -0.2073905 , -0.10017375, -5.76032828,
       -0.95325469, -0.25331492, -3.80041858,  0.70703991, -0.66220181,
        0.34091072,  0.43960581, -0.50383788])]
this is mape
[array([ 6.26193559,  6.46761488,  1.06472441,  7.22965826,  8.31596193,
        9.13443542,  8.02389196,  2.31622867,  3.00544389,  3.78072863,
        4.22206723,  8.24301825,  4.44

In [ ]:
test = [p for p in test if p not in ['ID00011637202177653955184', 'ID00052637202186188008618']]
osic_test = OSICData_test(test, A, TAB)
test_loader = torch.utils.data.DataLoader(osic_test, batch_size=1, num_workers=hyp.num_workers)


# load the best model
tabct = TabCT(cnn = model).to(gpu)
tabct.load_state_dict(torch.load(f"{result_dir}/september8th/{model}simplestep2hybrid.tar")["model_state_dict"])

running_loss = 0.0
pred_a = {}
tabct.eval()
tabular_loss = []
for i, data in enumerate(tqdm(test_loader, 0)):

    [mask, x, t], a, pid = data

    x = x.to(gpu)
    mask = mask.to(gpu)
    t = t.to(gpu)
    a = a.to(gpu)

    # forward
    outputs, tab_loss = tabct(x, t, mask)
    loss = criterion(outputs, a)
    tabular_loss.append(tab_loss)
    pids = pid
    preds_a = outputs.detach().cpu().numpy().flatten()
    print([outputs, pid])
    for j, p_d in enumerate(pids):
        pred_a[p_d] = preds_a[j]




    # print statistics
    running_loss += loss.item()
print(tabular_loss)
# print(f"epoch {epoch+1} val: {running_loss}")
# log.write(f"epoch {epoch+1} val: {running_loss}\n")
# score calculation
# print(pred_a)
# print(len(pred_a))
# print(p_test)
# print(len(p_test))

# totals
tot_r2_score = []
tot_mape_score = []
tot_mae_score = []
tot_smape_score = []

# everyone
score_v = 0.
rmse = 0.
test_r2_score = []
test_mape_score = []
test_mae_score = []
test_smape_score = []
# print(len(p_valid))
# fvcs_true = []
# fvcs_pred = []
for p in test:
    score_v += (score_avg(p, pred_a[p]))/len(test)
    rmse += (rmse_avg(p, pred_a[p]))/len(test)
    fvc_true = train.FVC.values[train.Patient == p]
    weeks_true = train.Weeks.values[train.Patient == p]
    fvc_pred = pred_a[p] * (weeks_true - weeks_true[0]) + fvc_true[0]

    test_r2_score.append(r2_score(fvc_true, fvc_pred))
    test_mape_score.append(np.mean(np.abs((fvc_true - fvc_pred) / fvc_true)) * 100)
    test_mae_score.append(np.mean(np.abs(fvc_pred - fvc_true)))
    test_smape_score.append(smape(fvc_true, fvc_pred))
#------------------------
tot_rmse.append(rmse)
tot_r2_score.append(np.asanyarray(test_r2_score))
tot_mape_score.append(np.asanyarray(test_mape_score))
tot_mae_score.append(np.asanyarray(test_mae_score))
tot_smape_score.append(np.asanyarray(test_smape_score))
#------------------------

print("this is rmse")
print(tot_rmse)
print("this is r2")
print(tot_r2_score)
print("this is mape")
print(tot_mape_score)
print("this is mae")
print(tot_mae_score)
print("this is smape")
print(tot_smape_score)
print(f"val score: {score_v}")
log.write(f"val score: {score_v}\n")
log.write(f"val rmse: {rmse}\n")

c:\Users\Vison\Documents\Users\Dolatabadi\Dolatabadi\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ViTHybridModel were not initialized from the model checkpoint at google/vit-hybrid-base-bit-384 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Vison\Documents\Users\Dolatabadi\Dolatabadi\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Vison\Documents\Users\Dolatabadi\Dolatabadi\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments oth

second feature extractor


  3%|▎         | 1/35 [00:01<00:46,  1.36s/it]

[tensor([[-4.9629]], device='cuda:0', grad_fn=<AddmmBackward0>), ('ID00025637202179541264076',)]


  6%|▌         | 2/35 [00:02<00:35,  1.08s/it]

...

 97%|█████████▋| 34/35 [00:28<00:00,  1.34it/s]

[tensor([[-5.4980]], device='cuda:0', grad_fn=<AddmmBackward0>), ('ID00407637202308788732304',)]


100%|██████████| 35/35 [00:29<00:00,  1.20it/s]

[tensor([[-4.5735]], device='cuda:0', grad_fn=<AddmmBackward0>), ('ID00123637202217151272140',)]
[tensor([1.0120], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9438], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9477], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.8658], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9694], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9567], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9329], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9822], device='cuda:0', grad_fn=<AddBackward0>), tensor([1.0142], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9850], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9477], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9587], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9328], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9616], device='cuda:0', grad_fn=<AddBackward0>), tensor([1.0336], device='cuda:0', grad_fn=<AddBackward0>), tensor([0.9741], 

29

In [25]:
print(pred_a)
print(test)

{'ID00025637202179541264076': -4.9629145, 'ID00417637202310901214011': -6.96957, 'ID00423637202312137826377': -5.1295543, 'ID00078637202199415319443': -8.249274, 'ID00042637202184406822975': -4.4956326, 'ID00210637202257228694086': -0.25509533, 'ID00367637202296290303449': -5.776394, 'ID00283637202278714365037': -6.7125416, 'ID00242637202264759739921': -4.6875415, 'ID00329637202285906759848': -4.974105, 'ID00111637202210956877205': -5.1242247, 'ID00135637202224630271439': -6.878868, 'ID00341637202287410878488': 0.0043280534, 'ID00371637202296828615743': -5.0871334, 'ID00048637202185016727717': -4.488312, 'ID00077637202199102000916': -6.0758114, 'ID00305637202281772703145': -4.573623, 'ID00331637202286306023714': -0.99795616, 'ID00110637202210673668310': -5.1420565, 'ID00319637202283897208687': -5.1295567, 'ID00336637202286801879145': -4.531871, 'ID00400637202305055099402': -4.3320036, 'ID00161637202235731948764': -4.636307, 'ID00364637202296074419422': -4.687541, 'ID0034463720228768421

In [ ]:
# final training with optimized setting

osic_all = OSICData_test(P, A, TAB)
all_loader = torch.utils.data.DataLoader(osic_all, batch_size=2, shuffle=True, num_workers=hyp.num_workers)

# load the best model
tabct = TabCT(cnn = model).to(gpu)
tabct.load_state_dict(torch.load(f"{result_dir}/september8th/{model}simplestep2hybrid.tar")["model_state_dict"])

optimizer = torch.optim.AdamW(tabct.parameters(), lr = hyp.final_lr) # very small learning rate


print(f"Final training")
log.write(f"Final training\n")  
for epoch in range(best_epoch + 2):  # loop over the dataset multiple times

    running_loss = 0.0
    tabct.train()
    for i, data in enumerate(tqdm(all_loader, 0)):

        [mask, x, t], a, _ = data

        x = x.to(gpu)
        mask = mask.to(gpu)
        t = t.to(gpu)
        a = a.to(gpu)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs, _ = tabct(x, t, mask)
        loss = criterion(outputs, a)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print(f"epoch {epoch+1} train: {running_loss}")
    log.write(f"epoch {epoch+1} train: {running_loss}\n")
    torch.save({
        'epoch': best_epoch,
        'model_state_dict': tabct.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
        }, f"{result_dir}/september8th/{model}simplestep2hybrid.tar")

print('Finished Training')
# destroy model
del tabct
torch.cuda.empty_cache()


# plot_figures(min_sq, 2, 3)

# ref: https://www.kaggle.com/miklgr500/linear-decay-based-on-resnet-cnn
# https://pytorch.org/docs/stable/index.html

c:\Users\Vison\Documents\Users\Dolatabadi\Dolatabadi\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ViTHybridModel were not initialized from the model checkpoint at google/vit-hybrid-base-bit-384 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Vison\Documents\Users\Dolatabadi\Dolatabadi\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Vison\Documents\Users\Dolatabadi\Dolatabadi\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments oth

second feature extractor
Final training


100%|██████████| 87/87 [01:53<00:00,  1.30s/it]


epoch 1 train: 377.20458340644836


100%|██████████| 87/87 [01:54<00:00,  1.31s/it]


...

epoch 5 train: 375.00669288635254


100%|██████████| 87/87 [01:48<00:00,  1.25s/it]


epoch 6 train: 378.1719209551811


100%|██████████| 87/87 [01:48<00:00,  1.24s/it]


epoch 7 train: 364.9025667309761
Finished Training


In [27]:
# load the best model
tabct = TabCT(cnn = model).to(gpu)
tabct.load_state_dict(torch.load(f"{result_dir}/september8th/{model}simplestep2hybrid.tar")["model_state_dict"])

Some weights of ViTHybridModel were not initialized from the model checkpoint at google/vit-hybrid-base-bit-384 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


second feature extractor


<All keys matched successfully>

# VIT

## CT

In [30]:
# C:\Users\Vison\Documents\Users\Dolatabadi\FibroCosaNet\train\ID00216637202257988213445\10.dcm
image = get_img("../train/ID00216637202257988213445/10.dcm")
image = np.stack((image, image, image), axis=-1)
image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0) .to(gpu)
out = tabct.conv(image).squeeze(0)
tensor = out  # Example: Replace with your actual tensor
print(type(tensor))
# Convert tensor to numpy if needed
tensor_np = tensor.to("cpu").detach().numpy()


# List of colormaps to visualize
colormaps = ['gray', 'bone', 'inferno', 'viridis', 'magma', 'jet', 'hot', 'coolwarm', 'plasma', 'cividis']

# Create a folder to save the images
save_dir = "slices_by_cmap/VIT/image"
os.makedirs(save_dir, exist_ok=True)

# Plot all slices for each colormap and save the output
for cmap in colormaps:
    # Create a figure with subplots
    fig, axes = plt.subplots(8, 8, figsize=(15, 15))  # 8x8 grid for 64 slices

    # Plot each slice in the grid
    for i in range(8):
        for j in range(8):
            slice_idx = i * 8 + j  # Calculate the current slice index
            axes[i, j].imshow(tensor_np[slice_idx], cmap=cmap)
            axes[i, j].set_title(f'Channel {slice_idx}')
            axes[i, j].axis('off')  # Hide the axis for clarity

    # Adjust layout
    plt.tight_layout()

    # Save the figure for the current colormap
    save_path = os.path.join(save_dir, f"channels_{cmap}.png")
    plt.savefig(save_path)

    # Close the figure to release memory
    plt.close(fig)

print(f"Images saved in directory: {save_dir}")

<class 'torch.Tensor'>
Images saved in directory: slices_by_cmap/VIT/image


## mask

In [31]:
# C:\Users\Vison\Documents\Users\Dolatabadi\FibroCosaNet\mask_clear\10.jpg
image = get_mask("../mask_clear/ID00216637202257988213445/10.jpg")
image = np.stack((image, image, image), axis=-1)
image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0) .to(gpu)
out = tabct.mask(image).squeeze(0)
tensor = out  # Example: Replace with your actual tensor
print(type(tensor))
# Convert tensor to numpy if needed
tensor_np = tensor.to("cpu").detach().numpy()


# List of colormaps to visualize
colormaps = ['gray', 'bone', 'inferno', 'viridis', 'magma', 'jet', 'hot', 'coolwarm', 'plasma', 'cividis']

# Create a folder to save the images
save_dir = "slices_by_cmap/VIT/mask"
os.makedirs(save_dir, exist_ok=True)

# Plot all slices for each colormap and save the output
for cmap in colormaps:
    # Create a figure with subplots
    fig, axes = plt.subplots(8, 8, figsize=(15, 15))  # 8x8 grid for 64 slices

    # Plot each slice in the grid
    for i in range(8):
        for j in range(8):
            slice_idx = i * 8 + j  # Calculate the current slice index
            axes[i, j].imshow(tensor_np[slice_idx], cmap=cmap)
            axes[i, j].set_title(f'Channel {slice_idx}')
            axes[i, j].axis('off')  # Hide the axis for clarity

    # Adjust layout
    plt.tight_layout()

    # Save the figure for the current colormap
    save_path = os.path.join(save_dir, f"Channels_{cmap}.png")
    plt.savefig(save_path)

    # Close the figure to release memory
    plt.close(fig)

print(f"Images saved in directory: {save_dir}")

<class 'torch.Tensor'>
Images saved in directory: slices_by_cmap/VIT/mask
